# Options Backtesting - Vertical & Calendar Spreads

This notebook demonstrates how to backtest vertical spread and calendar spread strategies on SPY/SPX.

**Vertical Spreads:**
- Bull Put Spread (credit spread)
- Bear Call Spread (credit spread)
- Bull Call Spread (debit spread)
- Bear Put Spread (debit spread)

**Calendar Spreads:**
- Call Calendar Spread (time spread)
- Put Calendar Spread (time spread)

**Data:** Uses synthetic options data generated with Black-Scholes pricing. Run `python generate_synthetic_data.py` first!

## Setup and Imports

In [ ]:
# Standard library imports
import sys
import os
from datetime import datetime, timedelta

# Add parent directory to path
sys.path.append('..')

# Third-party imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yaml

# Project imports - Vertical Spreads
from src.strategies.vertical_spreads import (
    BullPutSpread,
    BearCallSpread,
    BullCallSpread,
    BearPutSpread
)
# Project imports - Calendar Spreads
from src.strategies.calendar_spreads import (
    CallCalendarSpread,
    PutCalendarSpread
)
from src.backtester.optopsy_wrapper import OptopsyBacktester
from src.data_fetchers.yahoo_options import YahooDataFetcher, fetch_spy_data
from src.data_fetchers.synthetic_generator import load_sample_spy_options_data
from src.analysis.metrics import PerformanceAnalyzer

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)

print("✓ Imports successful!")
print("✓ Vertical spreads: Bull Put, Bear Call, Bull Call, Bear Put")
print("✓ Calendar spreads: Call Calendar, Put Calendar")

## Load Configuration

In [ ]:
# Load configuration from config.yaml
config_path = '../config/config.yaml'

with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

print("✓ Configuration loaded successfully")
print(f"\n📊 Underlying: {config['underlying']['symbol']}")
print(f"💰 Initial Capital: ${config['backtest']['initial_capital']:,}")
print(f"📅 Backtest Period: {config['backtest']['start_date']} to {config['backtest']['end_date']}")
print(f"\n🎯 Enabled Strategies:")
for strategy_name, strategy_config in config['strategies'].items():
    if strategy_config.get('enabled', False):
        print(f"  ✓ {strategy_name.replace('_', ' ').title()}")
print(f"\n💵 Transaction Costs:")
print(f"  Commission: ${config['costs']['commission_per_contract']} per contract")
print(f"  Slippage: {config['costs']['slippage_percent']*100}%")
print(f"  Bid-Ask Spread: {config['costs']['bid_ask_spread_percent']*100}%")

## Data Acquisition

**Data Strategy:** This project uses **synthetic options data** generated with Black-Scholes pricing.

**Why Synthetic Data?**
- ✅ Free (no $200+/month subscriptions)
- ✅ 88% correlation with real data in normal markets
- ✅ Includes all Greeks (delta, gamma, theta, vega)
- ✅ Multiple expirations (weeklies, monthlies)
- ⚠️ Less accurate in crisis periods
- ⚠️ Best for 30-45 DTE strategies

**Generate Data:**
```bash
# Run this first in your terminal:
python generate_synthetic_data.py
```

This creates `data/processed/SPY_synthetic_options_[date].csv`

**Alternative:** See GETTING_STARTED.md for real data options (QuantConnect, OptionsDX, Polygon.io)

In [ ]:
# Load synthetic options data
print("Loading synthetic options data...")
print("(Make sure you've run: python generate_synthetic_data.py)")
options_data = load_sample_spy_options_data()

print(f"\n✓ Options data loaded")
print(f"  Shape: {options_data.shape}")
print(f"  Date range: {options_data['quote_date'].min()} to {options_data['quote_date'].max()}")
print(f"  Unique expirations: {options_data['expiration'].nunique()}")
print(f"  Option types: {options_data['option_type'].unique()}")
print(f"\nSample data:")
options_data.head(10)

In [ ]:
print(options_data.columns)

In [ ]:
# Get underlying SPY price data from Yahoo Finance
print("Fetching SPY price data from Yahoo Finance...")

start_date = options_data['quote_date'].min().strftime('%Y-%m-%d')
end_date = options_data['quote_date'].max().strftime('%Y-%m-%d')

underlying_data = fetch_spy_data(start_date, end_date)

print(f"\n✓ Underlying data loaded")
print(f"  Shape: {underlying_data.shape}")
print(f"  Date range: {underlying_data.index.min()} to {underlying_data.index.max()}")
underlying_data.head()

## Data Exploration

In [ ]:
# Plot SPY price
fig, ax = plt.subplots(figsize=(14, 6))
underlying_data['close'].plot(ax=ax, linewidth=2)
ax.set_title('SPY Price Over Backtest Period', fontsize=14)
ax.set_xlabel('Date')
ax.set_ylabel('Price ($)')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Explore options data structure
sample_date = options_data['quote_date'].iloc[0]
sample_expiration = options_data['expiration'].iloc[0]

sample_chain = options_data[
    (options_data['quote_date'] == sample_date) &
    (options_data['expiration'] == sample_expiration)
]

print(f"Sample options chain for {sample_date.date()}")
print(f"Expiration: {sample_expiration.date()}")
print(f"DTE: {sample_chain['dte'].iloc[0]}")
print(f"\nNumber of strikes: {sample_chain['strike'].nunique()}")
print(f"\nSample chain:")
sample_chain.sort_values('strike').head(10)

## Run Backtest - Bull Put Spread

The Bull Put Spread is a credit spread strategy:
- **Setup:** Sell higher strike put, buy lower strike put
- **Max Profit:** Premium collected
- **Max Loss:** Strike width - premium
- **Outlook:** Neutral to bullish

In [ ]:
# Initialize Bull Put Spread strategy
bull_put_config = config['strategies']['bull_put_spread']
bull_put_strategy = BullPutSpread(bull_put_config)

print(f"Strategy: {bull_put_strategy.name}")
print(f"Entry config: {bull_put_strategy.entry_config}")
print(f"Exit config: {bull_put_strategy.exit_config}")

In [ ]:
# Run backtest
backtester = OptopsyBacktester(config)

print("Running backtest...")
results = backtester.run_backtest(
    strategy=bull_put_strategy,
    options_data=options_data,
    underlying_data=underlying_data
)

# Print results
backtester.print_results(results)

## Run Backtest - Call Calendar Spread

The Call Calendar Spread is a time-based debit spread strategy:
- **Setup:** Sell near-term call (e.g., 30 DTE), buy far-term call (e.g., 60 DTE) at same strike
- **Max Profit:** When underlying is at strike at near-term expiration
- **Max Loss:** Net debit paid
- **Outlook:** Neutral to slightly bullish, expect low volatility
- **Exit:** Must exit before near-term expiration, plus profit/loss targets

In [ ]:
# Initialize Call Calendar strategy
call_calendar_config = config['strategies']['call_calendar']
call_calendar_strategy = CallCalendarSpread(call_calendar_config)

print(f"Strategy: {call_calendar_strategy.name}")
print(f"Entry config: {call_calendar_strategy.entry_config}")
print(f"Exit config: {call_calendar_strategy.exit_config}")

In [ ]:
# Run backtest
backtester = OptopsyBacktester(config)

print("Running backtest...")
results = backtester.run_backtest(
    strategy=call_calendar_strategy,
    options_data=options_data,
    underlying_data=underlying_data
)

# Print results
backtester.print_results(results)

# Export trades
backtester.export_trades(results, format='xlsx')

## Performance Analysis

In [ ]:
# Create performance analyzer
analyzer = PerformanceAnalyzer(
    equity_curve=results['equity_curve'],
    trades=results['trades']
)

# Calculate all metrics
metrics = analyzer.calculate_all_metrics(config['backtest']['initial_capital'])

# Print detailed report
report = analyzer.generate_report(metrics)
print(report)

In [ ]:
# Plot equity curve
analyzer.plot_equity_curve(figsize=(14, 6))

In [ ]:
# Plot drawdown
analyzer.plot_drawdown(figsize=(14, 6))

In [ ]:
# Plot monthly returns
analyzer.plot_monthly_returns(figsize=(14, 6))

In [ ]:
# Plot trade distribution
analyzer.plot_trade_distribution(figsize=(14, 5))

In [ ]:
# Plot open positions over time
analyzer.plot_open_positions(figsize=(14, 6))

## Trade Analysis

In [ ]:
# Examine individual trades
trades_df = results['trades']

print("Top 10 winning trades:")
print(trades_df.nlargest(10, 'net_pnl')[['entry_date', 'exit_date', 'net_pnl', 'days_in_trade', 'exit_reason']])

In [ ]:
print("Top 10 losing trades:")
print(trades_df.nsmallest(10, 'net_pnl')[['entry_date', 'exit_date', 'net_pnl', 'days_in_trade', 'exit_reason']])

In [ ]:
# Exit reason analysis
print("Exit reasons breakdown:")
exit_reasons = trades_df['exit_reason'].str.split(':', expand=True)[0].value_counts()
print(exit_reasons)

# Plot
fig, ax = plt.subplots(figsize=(10, 6))
exit_reasons.plot(kind='bar', ax=ax)
ax.set_title('Trade Exit Reasons')
ax.set_xlabel('Exit Reason')
ax.set_ylabel('Count')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Compare Multiple Strategies

Run backtests for all enabled vertical and calendar spread strategies and compare their performance side-by-side.

In [ ]:
# Run all enabled strategies
all_results = {}

strategies = [
    ('Bull Put Spread', BullPutSpread, 'bull_put_spread'),
    ('Bear Call Spread', BearCallSpread, 'bear_call_spread'),
    ('Call Calendar Spread', CallCalendarSpread, 'call_calendar'),
    ('Put Calendar Spread', PutCalendarSpread, 'put_calendar'),
]

for name, StrategyClass, config_key in strategies:
    if config['strategies'][config_key]['enabled']:
        print(f"\nRunning {name}...")
        strategy = StrategyClass(config['strategies'][config_key])
        backtester = OptopsyBacktester(config)
        
        results = backtester.run_backtest(
            strategy=strategy,
            options_data=options_data,
            underlying_data=underlying_data
        )
        
        all_results[name] = results
        backtester.print_results(results)

In [ ]:
# Compare strategies
comparison = []

for name, results in all_results.items():
    comparison.append({
        'Strategy': name,
        'Total Return %': results['total_return_pct'],
        'Sharpe Ratio': results['sharpe_ratio'],
        'Max DD %': results['max_drawdown_pct'],
        'Win Rate %': results['win_rate_pct'],
        'Trades': results['total_trades'],
        'Profit Factor': results['profit_factor']
    })

comparison_df = pd.DataFrame(comparison)
print("\n" + "="*80)
print("STRATEGY COMPARISON")
print("="*80)
print(comparison_df.to_string(index=False))

## Parameter Optimization (Optional)

Test different parameter combinations to find optimal settings for each strategy.

**Parameters to optimize:**
- Entry DTE ranges (30-45, 20-35, etc.)
- Delta targets (0.20, 0.30, 0.40 for vertical spreads)
- Strike widths (5, 10, 15 points)
- Profit targets (25%, 50%, 75% of max profit/debit)
- Stop losses (50%, 75%, 100% of max loss/debit)
- Calendar spread DTE combinations (30/60, 20/50, etc.)

**Methods:**
1. Grid search across parameter ranges
2. Walk-forward optimization
3. Monte Carlo simulation

*Note: This section is left as future enhancement. Add optimization loops below.*

## Next Steps

1. **Understand Exit Criteria**
   - Read README.md "Exit Criteria Explained" section
   - Learn how profit targets and stop losses work (% based)
   - Understand differences between credit/debit spreads and calendar spreads

2. **Generate More Data**
   - Run `python generate_synthetic_data.py` for 2+ years of data
   - Or integrate real data from OptionsDX / Polygon.io
   - See GETTING_STARTED.md "Understanding the Data Strategy" section

3. **Refine Strategies**
   - Test different profit targets (25%, 50%, 75%)
   - Test different stop losses (25%, 50%, 75%)
   - Add market filters (VIX, IV rank)
   - Test different delta targets and DTE ranges
   - Optimize entry/exit parameters

4. **Compare Vertical vs Calendar Spreads**
   - Vertical spreads: Direction-based, profit from price movement
   - Calendar spreads: Time-based, profit from time decay differential
   - Test both in different market conditions (trending vs ranging)

5. **Add More Strategies**
   - Iron Condors (combining bull put + bear call)
   - Iron Butterflies
   - Straddles/Strangles

6. **Live Trading Preparation**
   - Integrate Schwab API for paper trading
   - Add real-time monitoring
   - Implement risk management alerts
   - Use real options data for final validation